In [ ]:
!pip install neptune

In [2]:
import os
import json
import torch
import pandas as pd
import matplotlib.pyplot as plt
from fastai.vision.all import *
from timm import create_model
import neptune
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# === Pfade definieren ===
DATASET_DIR = "/kaggle/input/wildlife-trap-images/balanced_dataset_split(ohne aves)/balanced_dataset_split"
JSON_PATH = "/kaggle/input/wildlife-trap-images/train_test_split_new.json"
OUTPUT_DIR = "/kaggle/working/model_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# === JSON mit Train/Test-Split laden ===
with open(JSON_PATH, "r") as f:
    split_data = json.load(f)

df_train = pd.DataFrame(split_data["train"])
df_test = pd.DataFrame(split_data["test"])

df_train["image_path"] = df_train["file_path"].apply(lambda x: os.path.join(DATASET_DIR, x))
df_test["image_path"] = df_test["file_path"].apply(lambda x: os.path.join(DATASET_DIR, x))

print(f"Trainingsdaten: {len(df_train)} Bilder, Testdaten: {len(df_test)} Bilder")

print(df_train["label"].unique().tolist())
print(len(df_train["label"].unique().tolist()))

✅ Trainingsdaten: 7900 Bilder, Testdaten: 1991 Bilder
['acinonyx jubatus', 'antidorcas marsupialis', 'canis mesomelas', 'cn-francolins', 'cn-owls', 'cn-raptors', 'columbidae', 'corvus albus', 'corvus capensis', 'crocuta crocuta', 'diceros bicornis', 'equus asinus', 'equus zebra hartmannae', 'eupodotis rueppellii', 'giraffa camelopardalis', 'hyaena brunnea', 'hystrix africaeaustralis', 'lepus capensis', 'loxodanta africana', 'mellivora capensis', 'neotis ludwigii', 'numididae', 'oreotragus oreotragus', 'oryx gazella', 'otocyon megalotis', 'panthera leo', 'panthera pardus', 'papio anubis', 'procavia capensis', 'pronolagus randensis', 'pteroclidae', 'raphiceros campestris', 'struthio camelus', 'torgos tracheliotos', 'tragelaphus strepsiceros', 'vulpes chama']
36


In [ ]:
EXTRA_IMAGES_DIR = "/kaggle/input/wildlife-trap-images/augmented_dataset_new(ohne aves)/augmented_dataset_new" # Augmentierte Bilder

INAT_IMAGES = "/kaggle/input/wildlife-trap-images/inat_images/inat_images"

# Neue Bilder sammeln
extra_data = []

# Durch alle Unterordner in EXTRA_IMAGES_DIR iterieren
for category in os.listdir(EXTRA_IMAGES_DIR):
    category_path = os.path.join(EXTRA_IMAGES_DIR, category)
    
    # Nur Ordner berücksichtigen (keine einzelnen Dateien)
    if os.path.isdir(category_path):
        image_files = [f for f in os.listdir(category_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for img in image_files:
            extra_data.append({
                "file_path": os.path.join(category_path, img),
                "label": category
            })

# NAT_IMAGES ergänzen
for category in os.listdir(INAT_IMAGES):
    category_path = os.path.join(INAT_IMAGES, category)
    
    if os.path.isdir(category_path):
        image_files = [f for f in os.listdir(category_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for img in image_files:
            extra_data.append({
                "file_path": os.path.join(category_path, img),
                "label": category
            })
            
# Neuen DataFrame für zusätzliche Bilder erstellen
df_extra = pd.DataFrame(extra_data)
df_extra["image_path"] = df_extra["file_path"]

# "_" durch " " in den Labels des neuen Datensatzes ersetzen
df_extra["label"] = df_extra["label"].str.replace("_", " ")

# Trainings-DataFrame mit den korrigierten Labels aktualisieren
df_train = pd.concat([df_train, df_extra], ignore_index=True)

print(df_extra["label"].unique())

print(f"Nach Ergänzung: {len(df_train)} Trainingsbilder")

['vulpes chama' 'cn-owls' 'numididae' 'lepus capensis' 'corvus capensis'
 'tragelaphus strepsiceros' 'equus asinus' 'oreotragus oreotragus'
 'cn-francolins' 'procavia capensis' 'columbidae' 'otocyon megalotis'
 'mellivora capensis' 'eupodotis rueppellii' 'papio anubis'
 'raphiceros campestris' 'pronolagus randensis']
✅ Nach Ergänzung: 12937 Trainingsbilder


In [5]:
# Ensure at least one image per category in df_train_sampled
min_per_category = 2
df_train_sampled = df_train.groupby('label', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), min_per_category), random_state=42)
).reset_index(drop=True)

# Fill up to 100 samples if more data is available
remaining_samples = 100 - len(df_train_sampled)
if remaining_samples > 0:
    df_remaining = df_train[~df_train.index.isin(df_train_sampled.index)]
    if len(df_remaining) >= remaining_samples:  # Ensure we have enough samples
        df_train_sampled = pd.concat([df_train_sampled, df_remaining.sample(n=remaining_samples, random_state=42)])
    else:
        df_train_sampled = pd.concat([df_train_sampled, df_remaining])  # Take all remaining samples

# Ensure at least one image per category in df_test_sampled
df_test_sampled = df_test.groupby('label', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 1), random_state=42)
).reset_index(drop=True)

# Vokabular aus Labels erstellen
vocab = df_train["label"].unique().tolist()
#vocab = df_train_sampled["label"].unique().tolist()

print(df_train["label"].unique().tolist())
print(len(df_train["label"].unique().tolist()))


['acinonyx jubatus', 'antidorcas marsupialis', 'canis mesomelas', 'cn-francolins', 'cn-owls', 'cn-raptors', 'columbidae', 'corvus albus', 'corvus capensis', 'crocuta crocuta', 'diceros bicornis', 'equus asinus', 'equus zebra hartmannae', 'eupodotis rueppellii', 'giraffa camelopardalis', 'hyaena brunnea', 'hystrix africaeaustralis', 'lepus capensis', 'loxodanta africana', 'mellivora capensis', 'neotis ludwigii', 'numididae', 'oreotragus oreotragus', 'oryx gazella', 'otocyon megalotis', 'panthera leo', 'panthera pardus', 'papio anubis', 'procavia capensis', 'pronolagus randensis', 'pteroclidae', 'raphiceros campestris', 'struthio camelus', 'torgos tracheliotos', 'tragelaphus strepsiceros', 'vulpes chama']
36


<ipython-input-5-be42cec4a291>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train_sampled = df_train.groupby('label', group_keys=False).apply(
<ipython-input-5-be42cec4a291>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_test_sampled = df_test.groupby('label', group_keys=False).apply(


In [6]:
print("Länge Vocab: ", len(vocab))

Länge Vocab:  36


In [ ]:
batch_sizes = [64]

for bs in batch_sizes:
    print(f"\n🚀 Starte Training mit Batch Size = {bs}")
    
    # === Neptune-Run initialisieren ===
    run = neptune.init_run(
        project="katzn13/DL-models",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5OWI0ZWI1Yy0wNzM5LTRhOTEtODk5Yi1mNTUyNWQwOTkxMWUifQ==",
        tags=[f"ViT-bs{bs}+augm+iNat"]
    )

    # === DataBlock und Dataloaders ===
    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock(vocab=vocab)),
        get_x=ColReader('image_path'),
        get_y=ColReader('label'),
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        item_tfms=Resize(224),
    )

    dls = dblock.dataloaders(df_train, bs=bs, num_workers=8, device="cuda")
    test_dl = dls.test_dl(df_test, device="cuda", with_labels=True)

    # === ViT-Modell vorbereiten ===
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    vit_model = create_model("vit_base_patch16_224", pretrained=True, num_classes=len(vocab)).to(device)

    learn = Learner(
        dls,
        vit_model,
        loss_func=CrossEntropyLossFlat(),
        metrics=accuracy,
        cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
    ).to_fp16()

    # === Neptune Parameter loggen ===
    run["parameters"] = {
        "dataset_dir": DATASET_DIR,
        "batch_size": bs,
        "epochs": 100,
        "architecture": "ViT (vit_base_patch16_224)",
    }

    # === Lernrate finden und loggen ===
    lr_find_results = learn.lr_find(start_lr=1e-6, end_lr=1e-2, show_plot=False)
    optimal_lr = lr_find_results.valley
    run["learning_rate/optimal"] = optimal_lr
    print(f"Empfohlene Lernrate (bs={bs}): {optimal_lr}")

    # === Training ===
    learn.fit_one_cycle(100, lr_max=optimal_lr)

    # === Losses loggen ===
    for loss in learn.recorder.losses:
        run["train/loss"].append(loss.item())
    for values in learn.recorder.values:
        if len(values) > 1:
            run["valid/loss"].append(values[1])
        if len(values) > 2:
            run["valid/accuracy"].append(values[2])

    # === Modell speichern ===
    model_path = f"{OUTPUT_DIR}/vit_model_bs{bs}.pkl"
    learn.export(model_path)
    run["model"].upload(model_path)

    # === Testevaluation ===
    test_loss, test_acc = learn.validate(dl=test_dl)
    run["test/loss"] = test_loss
    run["test/accuracy"] = test_acc
    print(f"Test Accuracy für bs={bs}: {test_acc:.4f}")

    # === Neptune Run beenden ===
    run.stop()


🚀 Starte Training mit Batch Size = 64


[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/katzn13/DL-models/e/DLMOD-120


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


...

🧠 Empfohlene Lernrate (bs=64): 3.0199516913853586e-05


epoch,train_loss,valid_loss,accuracy,time
0,2.964745,2.679828,0.340162,09:39
1,1.839962,1.597191,0.576730,09:38
2,1.126267,1.076298,0.720139,09:38
3,0.739322,0.796284,0.783147,09:35
4,0.523786,0.654173,0.824121,09:35
5,0.346473,0.569968,0.847700,09:49
6,0.258018,0.551218,0.851179,09:47
7,0.187065,0.516089,0.862389,09:36
8,0.147573,0.528141,0.856204,09:33
9,0.132801,0.509599,0.863935,09:41


No improvement since epoch 9: early stopping


/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


✅ Test Accuracy für bs=64: 0.9021
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 2 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/katzn13/DL-models/e/DLMOD-120/metadata
